### Step 1: Join, Transform, and Standardize Data
This script loads in the Google data and other city- and country-level variables.

Most of the code here deals with missing observations, standardizing the data to account for missing modes or in/outbound data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

### Process city-level data

In [ ]:
year=2023

# load in google data, which should be in a subfolder called EIE_data_raw
# for access, see the Data Availability statement in the published paper
features = pd.read_csv("../../EIE_data_raw/features.csv",usecols=['feature_id','city','state','country','centroid','population'])
transport = pd.read_csv("../../EIE_data_raw/transport.csv")

# modes are cycling, walking, automobile, and motorcycle (extra modes are transit modes - bus, rail, tram, subway, and ferry)
# for most of the analysis, the denominator excludes the extra_modes
modes_of_transportation = ['automobile','motorcycle','cycling','on_foot']
extra_modes = ['bus','ferry','rail','subway','tram']

# auto km and bus km are in terms of vehicle miles and vehicle trips, so convert these to passenger
# these are google defaults for the globe. 
for mode, occupancy in zip(['automobile','bus'],[1.7, 9.9]):
    for bound in ['in','out','intra']:
        for m in ['trips','full_distance_km']:
            transport[f'{mode}_{bound}bound_{m}'] = transport[f'{mode}_{bound}bound_{m}'] * occupancy

# determine level of completeness
for mode in modes_of_transportation:
    missing_inbound_outbound = ((transport[f'{mode}_inbound_trips'].isnull()) | (transport[f'{mode}_outbound_trips'].isnull())) & (transport[f'{mode}_intrabound_trips'].notnull())
    missing_intrabound = (transport[f'{mode}_inbound_trips'].notnull()) & (transport[f'{mode}_outbound_trips'].notnull()) & (transport[f'{mode}_intrabound_trips'].isnull())
    complete = (transport[f'{mode}_inbound_trips'].notnull()) & (transport[f'{mode}_outbound_trips'].notnull()) & (transport[f'{mode}_intrabound_trips'].notnull())
    transport[f'{mode}_completeness'] = np.where(missing_inbound_outbound, 'missing in/bound outbound', (np.where(missing_intrabound, 'missing intrabound', np.where(complete, 'complete','no data'))))

# calculate total trips per mode
# In- and out-bound trips are weighted at 50%. See the published paper
for mode in modes_of_transportation:
    transport[f'{mode}_inbound_trips_adj']=transport[f'{mode}_inbound_trips']/2
    transport[f'{mode}_outbound_trips_adj']=transport[f'{mode}_outbound_trips']/2
    transport[mode+'_total_trips']=transport[[f'{mode}_intrabound_trips',
                                              f'{mode}_inbound_trips_adj',f'{mode}_outbound_trips_adj']].sum(axis=1)

# calculate total trips across our chosen denominator (no bus, rail, tram, subway, ferry)
transport['total_trips'] = transport[[f'{mode}_total_trips' for mode in modes_of_transportation]].sum(axis=1)
transport['intrabound_trips'] = transport[[f'{mode}_intrabound_trips' for mode in modes_of_transportation]].sum(axis=1)

# drop places with literally no trips
transport=transport[transport.total_trips>0]

# calculate percentages, and fill in missing intrabound
transport.cycling_intrabound_trips = transport.cycling_intrabound_trips.fillna(0)
transport.motorcycle_intrabound_trips = transport.motorcycle_intrabound_trips.fillna(0)

for mode in modes_of_transportation:
    transport[f'tripshare_{mode}_total']=transport.apply(lambda x: x[f'{mode}_total_trips']/x['total_trips'],axis=1)
    transport[f'tripshare_{mode}_intrabound']=transport.apply(lambda x: x[f'{mode}_intrabound_trips']/x['intrabound_trips'] if x['intrabound_trips']>0 else 0,axis=1)

# determine which modeshare basis will be used for modeshare calculations
full_trips=transport.apply(lambda x: True if (x['automobile_completeness'] == 'complete' and x['on_foot_completeness'] == 'complete') else False, axis=1)
intrabound_only=transport.apply(lambda x: True if ((x['automobile_completeness'] == 'missing in/bound outbound' or x['automobile_completeness'] == 'complete') & (x['on_foot_completeness'] == 'missing in/bound outbound' or x['on_foot_completeness'] == 'complete')) else False, axis=1)
transport['modeshare_basis'] = np.where(full_trips, 'full_trips', (np.where(intrabound_only, 'intrabound_only', 'drop')))

# drop places that don't even have complete intrabound data
transport=transport[transport['modeshare_basis']!='drop']

for index, row in transport.iterrows():
    if row['modeshare_basis'] == 'full_trips':
        transport.at[index, 'tripshare_cycling'] = row['tripshare_cycling_total']
        transport.at[index, 'tripshare_on_foot'] = row['tripshare_on_foot_total'] 
    elif row['modeshare_basis'] == 'intrabound_only':
        transport.at[index, 'tripshare_cycling'] = row['tripshare_cycling_intrabound']
        transport.at[index, 'tripshare_on_foot'] = row['tripshare_on_foot_intrabound']

transport["tripshare_bikewalk"] = transport["tripshare_cycling"] + transport["tripshare_on_foot"]

# this creates columns that stores either all_trips or intrabound_trips based on modeshare_basis
mask = transport.modeshare_basis == 'intrabound_only' 
for mode in modes_of_transportation:
    transport[f'trips_{mode}_touse']=transport[f'{mode}_total_trips']
    transport.loc[mask, f'trips_{mode}_touse']=transport[f'{mode}_intrabound_trips']

# create column of distance in km for all modes (including transit)
transport.cycling_intrabound_full_distance_km = transport.cycling_intrabound_full_distance_km.fillna(0)
transport.motorcycle_intrabound_full_distance_km =transport.motorcycle_intrabound_full_distance_km.fillna(0)
for mode in modes_of_transportation+extra_modes:
    # distance
    transport[f'km_{mode}']=transport[f'{mode}_intrabound_full_distance_km'].fillna(0)
    transport[f'km_{mode}']+=transport[f'{mode}_inbound_full_distance_km'].fillna(0) * 0.5
    transport[f'km_{mode}']+=transport[f'{mode}_outbound_full_distance_km'].fillna(0) * 0.5
    # just use intrabound for some cities
    transport.loc[mask, f'km_{mode}']=transport[f'{mode}_intrabound_full_distance_km'].fillna(0)
    
# for mode share maps, add in public transport
transport['intrabound_trips_allmodes'] = transport['intrabound_trips']
transport['trips_total_touse']=transport['total_trips']
for mode in extra_modes:
    transport['trips_total_touse']+=transport[mode+'_intrabound_trips'].fillna(0)
    transport['trips_total_touse']+=transport[mode+'_inbound_trips'].fillna(0)*0.5
    transport['trips_total_touse']+=transport[mode+'_outbound_trips'].fillna(0)*0.5
    transport['intrabound_trips_allmodes']+=transport[mode+'_intrabound_trips'].fillna(0)
    
# calculate km
transport.loc[mask, 'trips_total_touse']=transport['intrabound_trips_allmodes']
km_cols = ['km_'+mode for mode in modes_of_transportation+extra_modes]
transport['km_total'] = transport[km_cols].sum(axis=1)

# calculate total km on transit modes
km_cols_transit = ['km_'+mode for mode in extra_modes]
transport['km_transit'] = transport[km_cols_transit].sum(axis=1)

# create column of km-share for just main modes (i.e., no transit)
km_share_cols = ['km_'+mode for mode in modes_of_transportation]
transport['km_no_transit'] = transport[km_share_cols].sum(axis=1)
for mode in modes_of_transportation:
    transport[f'km_share_{mode}']=transport[f'km_{mode}']/transport['km_total']
    transport[f'km_share_{mode}_no_transit']=transport[f'km_{mode}']/transport['km_no_transit']


# create df of modeshare for year
dfs = {}
dfs[f'transport_{year}']=transport[transport.year==year][['feature_id','modeshare_basis','tripshare_cycling','tripshare_on_foot','tripshare_bikewalk',
                                                'trips_automobile_touse', 'km_automobile','km_share_automobile','km_share_automobile_no_transit',
                                                'trips_motorcycle_touse', 'km_motorcycle','km_share_motorcycle','km_share_motorcycle_no_transit',
                                                'trips_cycling_touse', 'km_cycling','km_share_cycling','km_share_cycling_no_transit',
                                                'trips_on_foot_touse', 'km_on_foot','km_share_on_foot','km_share_on_foot_no_transit',
                                                'km_bus','km_ferry', 'km_rail','km_subway','km_tram','km_transit',
                                                'trips_total_touse','km_total','km_no_transit']].copy()
dfs[f'transport_{year}'].rename(columns={'tripshare_cycling':f'tripshare_cycling_{str(year)[2:]}','tripshare_on_foot':f'tripshare_on_foot_{str(year)[2:]}','tripshare_bikewalk':f'tripshare_bikewalk_{str(year)[2:]}'},inplace=True)

# turn modeshare tag into a boolean integer
dfs[f'transport_{year}'].modeshare_basis=dfs[f'transport_{year}'].modeshare_basis.str.replace('intrabound_only','0').replace('full_trips','1')
dfs[f'transport_{year}'].rename(columns={'modeshare_basis':'includes_inboundoutbound'},inplace=True)
dfs[f'transport_{year}'].includes_inboundoutbound=dfs[f'transport_{year}'].includes_inboundoutbound.astype('int64')

In [ ]:
# add in whether there were any trips on high-capacity transit (rail/subway/tram) in year
hc_transit=transport.copy()
hc_transit=hc_transit[hc_transit.year==year]
transit_modes=['rail','subway','tram']
transit_col=[mode+'_inbound_trips' for mode in transit_modes]+[mode+'_outbound_trips' for mode in transit_modes]+[
    mode+'_intrabound_trips' for mode in transit_modes]
hc_transit['hc_transit_trips']=hc_transit[transit_col].sum(axis=1)
hc_transit['rail_in_city']=hc_transit['hc_transit_trips'].apply(lambda x: 1 if x>0 else 0)
hc_transit['rail_in_city']=hc_transit['rail_in_city'].astype('int64')

dfs[f'transport_{year}']=dfs[f'transport_{year}'].set_index('feature_id').join(hc_transit.set_index('feature_id')[['rail_in_city']]).reset_index()

In [ ]:
# join together all city-level data
# these data files are in the git repository
climate_metrics=pd.read_csv("data/climate_metrics.csv").drop(columns=['Unnamed: 0'])
street_metrics=pd.read_csv("data/street_metrics.tsv",sep='\t')
overlapped=pd.read_csv("data/polygon_matches.csv").drop(columns=['Unnamed: 0']) # used to drop cities where our boundaries do not match
dfs['data_'+str(year)[2:]]=features.set_index('feature_id').join(dfs[f'transport_{year}'].set_index('feature_id')).join(climate_metrics.set_index('feature_id')).join(
    street_metrics.set_index('feature_id')).join(overlapped.set_index('feature_id')).reset_index()

# drop anything with null values (places with no boundary at all or no bike/ped data for that year) or where the polygons don't match
dfs['data_'+str(year)[2:]]=dfs['data_'+str(year)[2:]].dropna(subset=['bikelane_length_km','mway_length_km','roads_km','slope','wt_density','sndi','sndi_difference'])
dfs['data_'+str(year)[2:]]=dfs['data_'+str(year)[2:]].dropna(subset=[f'tripshare_bikewalk_{str(year)[2:]}'])
dfs['data_'+str(year)[2:]]=dfs['data_'+str(year)[2:]][dfs['data_'+str(year)[2:]].contained!=True]

# keep only cities with a population > 0
dfs['data_'+str(year)[2:]]=dfs['data_'+str(year)[2:]][dfs['data_'+str(year)[2:]].population>0]

In [ ]:
# calculate bikelanes per km of roads
dfs['data_'+str(year)[2:]]['bikelane_per_road_km']=dfs['data_'+str(year)[2:]].bikelane_length_km/dfs['data_'+str(year)[2:]].roads_km
dfs['data_'+str(year)[2:]]['mway_per_road_km']=dfs['data_'+str(year)[2:]].mway_length_km/dfs['data_'+str(year)[2:]].roads_km

# calculate sndi (sndi_difference with floor of 0)
dfs['data_'+str(year)[2:]]['sndi_added']=dfs['data_'+str(year)[2:]].sndi_difference.apply(lambda x: x if x>0 else 0)

# log transform right-skew data
dfs['data_'+str(year)[2:]]['ln_population']=dfs['data_'+str(year)[2:]].population.apply(lambda x: np.log(x))
dfs['data_'+str(year)[2:]]['ln_sndi']=dfs['data_'+str(year)[2:]].sndi.apply(lambda x: np.log(x+1))
dfs['data_'+str(year)[2:]]['ln_wt_density']=dfs['data_'+str(year)[2:]].wt_density.apply(lambda x: np.log(x+1))
dfs['data_'+str(year)[2:]]['ln_slope']=dfs['data_'+str(year)[2:]].slope.apply(lambda x: np.log(x))
dfs['data_'+str(year)[2:]]['ln_sndi_added']=dfs['data_'+str(year)[2:]].sndi_added.apply(lambda x: np.log(x+1))

In [ ]:
# dictionary that maps core variable name to column name from original dataset
city_var_dict={'sndi':'ln_sndi','density':'ln_wt_density','precip':'ln_total_precipitation','min_temp':'min_temp',
               'max_temp':'max_temp','bikelanes':'bikelane_per_road_km','motorways':'mway_per_road_km','slope':'ln_slope',
               'sndi_added':'ln_sndi_added','population':'ln_population'}

# created dataframe of scaled x-variables
df_scaled=pd.DataFrame(preprocessing.scale(dfs['data_'+str(year)[2:]][[value for value in city_var_dict.values()]]),
                       columns=[key+'_standard' for key in city_var_dict.keys()],index=dfs['data_'+str(year)[2:]].index)

# create square values of temperature data and density (for polynomial relationship to dependent)
df_scaled['min_temp_standard2']=df_scaled['min_temp_standard']**2
df_scaled['max_temp_standard2']=df_scaled['max_temp_standard']**2
df_scaled['density_standard2']=df_scaled['density_standard']**2

# reverse the direction of sndi (so that it measures connectedness instead of disconnectedness)
df_scaled['sndi_standard']=-df_scaled['sndi_standard']

dfs['data_'+str(year)[2:]] = pd.concat([dfs['data_'+str(year)[2:]], df_scaled], axis=1)

### Process country-level data

In [ ]:
# create df with list of countries
country_data = pd.DataFrame(dfs['data_'+str(year)[2:]].sort_values('country').country.unique(), columns = ['country'])

country_data['country_num'] = country_data.index

# drop "N/A" result of cities in places not assigned a country (e.g. in Kashmir)
country_data=country_data.dropna(subset=['country']) 

# open WDI data and clean up names so will join to google names
wdi=pd.read_csv('data/wdi.csv')
name_dict={'Brunei Darussalam':'Brunei',
           'Curacao':'Curaçao',
           "Cote d'Ivoire":"Côte d'Ivoire",
           "Congo, Dem. Rep.":"Democratic Republic of the Congo",
           "Egypt, Arab Rep.":"Egypt",
           "Iran, Islamic Rep.":"Iran",
           "Kyrgyz Republic":"Kyrgyzstan",
           "Lao PDR":"Laos",
           "Myanmar":"Myanmar (Burma)",
           "Congo, Rep.":"Republic of the Congo",
           "Slovak Republic":"Slovakia",
           "Syrian Arab Republic":"Syria",
           "Bahamas, The":"The Bahamas",
           "Gambia, The":"The Gambia",
           "Turkiye":"Turkey",
           "Venezuela, RB":"Venezuela",
           "Yemen, Rep.":"Yemen"}
wdi['country']=wdi['Country Name'].replace(name_dict)

# transform numeric data to be numeric
numeric_cols = [col for col in wdi.columns.to_list() if col.endswith(']')]
for col in numeric_cols:
    wdi[col] = pd.to_numeric(wdi[col], errors='coerce')

# find the most recent value for each metric
wdi['most_recent_value']=wdi.apply(lambda x: x['2022 [YR2022]'],axis=1)
for y in [2021,2020,2019,2018]:
    wdi['most_recent_value']=wdi.apply(lambda x: x.most_recent_value if x.most_recent_value >=0 else x[f'{y} [YR{y}]'], axis=1)

# pivot to transform dataset from one row per variable to one row per country with variable's most recent value
wdi_country=pd.pivot_table(wdi,index='country',values='most_recent_value',columns='Series Code',aggfunc='first').reset_index()

# do join 
country_data=country_data.set_index('country').join(wdi_country.set_index('country'),how='left').reset_index()

Semi-manual process of filling in missing data that are missing from the WDI data or that correspond to a geography that does not correspond well to the WDI geographies before joining this data to the city-level data.
* The Google EIE data treats French oversees departments (French Guiana, Guadeloupe, Martinique, and Réunion) as their own countries, while the WDI data only report on these geographies as being part of France. Given that the characteristics of the oversees departments differ substantially from mainland France, we used data from Insee (the French National Institute for Statistics and Economic Studies) to [calculate adjustment factors](https://docs.google.com/spreadsheets/d/1id_FmPEmkoihhck7v55YmwrFKaGzNShf/edit?usp=drive_link&ouid=110519951430869187067&rtpof=true&sd=true) for each oversees department, and then estimated department-level values by applying these adjustment factors to the most recently available WDI data for France. [Regional GDP per capita](https://www.insee.fr/fr/statistiques/5020211) were used to adjust GDP per capita, and [base population statistics](https://www.insee.fr/fr/statistiques/2521169) were used to adjust the dependency ratio (with this [Territory Comparator](https://www.insee.fr/fr/statistiques/1405599?geo=REG-01+REG-02+REG-03+REG-04) used to identify the regional codes associated with each oversees department). New Caledonia was assigned average oversees department values.
* Other missing data were replaced based on regional averages (e.g. data for Venezuela were substituted with data for the "Latin America & Carribean" region and data for Taiwan were substituted with data for the "East Asia & Pacific" region).

In [ ]:
# replace data for France.
france_gdp={'French Guiana':0.423098983*country_data[country_data.country=='France']['NY.GDP.PCAP.KD'].iloc[0],
                 'Guadeloupe':0.635527723*country_data[country_data.country=='France']['NY.GDP.PCAP.KD'].iloc[0],
                 'Martinique':0.693928145*country_data[country_data.country=='France']['NY.GDP.PCAP.KD'].iloc[0],
                 'Réunion':0.634817101*country_data[country_data.country=='France']['NY.GDP.PCAP.KD'].iloc[0],
                 'New Caledonia':0.567267473*country_data[country_data.country=='France']['NY.GDP.PCAP.KD'].iloc[0]}
france_dep={'French Guiana':1.018224798*country_data[country_data.country=='France']['SP.POP.DPND'].iloc[0],
                 'Guadeloupe':0.987761253*country_data[country_data.country=='France']['SP.POP.DPND'].iloc[0],
                 'Martinique':0.973945071*country_data[country_data.country=='France']['SP.POP.DPND'].iloc[0],
                 'Réunion':0.867912461*country_data[country_data.country=='France']['SP.POP.DPND'].iloc[0],
                 'New Caledonia':0.933531615*country_data[country_data.country=='France']['SP.POP.DPND'].iloc[0]}
for key,value in france_gdp.items():
    country_data.loc[country_data.country==key, 'NY.GDP.PCAP.KD'] = value    
for key,value in france_dep.items():
    country_data.loc[country_data.country==key, 'SP.POP.DPND'] = value

# replace critical missing values with their regional averages
country_data['NY.GDP.PCAP.KD']=country_data.apply(
    lambda x: wdi_country[wdi_country.country=='Latin America & Caribbean']['NY.GDP.PCAP.KD'].iloc[0] if x.country=='Venezuela' else x['NY.GDP.PCAP.KD'],axis=1)
country_data['NY.GDP.PCAP.KD']=country_data.apply(
    lambda x: wdi_country[wdi_country.country=='Africa Eastern and Southern']['NY.GDP.PCAP.KD'].iloc[0] if x.country=='Eritrea' else x['NY.GDP.PCAP.KD'],axis=1)
country_data['NY.GDP.PCAP.KD']=country_data.apply(
    lambda x: wdi_country[wdi_country.country=='United Kingdom']['NY.GDP.PCAP.KD'].iloc[0] if x.country=='Gibraltar' else x['NY.GDP.PCAP.KD'],axis=1)

country_data['NY.GDP.PCAP.KD']=country_data.apply(
    lambda x: wdi_country[wdi_country.country=='East Asia & Pacific']['NY.GDP.PCAP.KD'].iloc[0] if x.country=='Taiwan' else x['NY.GDP.PCAP.KD'],axis=1)    
country_data['SP.POP.DPND']=country_data.apply(
    lambda x: wdi_country[wdi_country.country=='East Asia & Pacific']['SP.POP.DPND'].iloc[0] if x.country=='Taiwan' else x['SP.POP.DPND'],axis=1)

# add gasoline prices
fuel=pd.read_csv('data/gasoline_prices_2018.csv')
country_data=country_data.set_index('country').join(fuel.set_index('country')).reset_index()

# log transform gdp
country_data['ln_gdp']=country_data['NY.GDP.PCAP.KD'].apply(lambda x: np.log(x+1))

# created dataframe of scaled x-variables
country_scaled=pd.DataFrame(preprocessing.scale(country_data[['ln_gdp','SP.POP.DPND','gasoline_2018']]),
                       columns=['gdp_standard','dependency_standard','gasoline_standard'],index=country_data.country).join(country_data.set_index('country')[['country_num','ln_gdp','SP.POP.DPND','gasoline_2018']])


# add in country-level mode-shares
country_shares= dfs['data_'+str(year)[2:]].groupby('country')[['trips_automobile_touse','trips_motorcycle_touse','trips_cycling_touse', 'trips_on_foot_touse','trips_total_touse',
                                            'km_on_foot','km_motorcycle','km_automobile','km_cycling','km_total','km_no_transit','population']].sum()
for mode in modes_of_transportation:
    country_shares[mode+'_share_including_transit'] = country_shares['trips_'+mode+'_touse'] / country_shares.trips_total_touse
    country_shares[f'km_share_{mode}'] = country_shares[f'km_{mode}'] / country_shares.km_total
    country_shares[f'km_share_{mode}_no_transit'] = country_shares[f'km_{mode}'] / country_shares.km_no_transit
cols_to_join = [col for col in country_shares.columns if '_share' in col or 'km_' in col or 'population' in col]
country_scaled = country_scaled.join(country_shares[cols_to_join])

# calculate km per capita for each mode
for col in ['km_on_foot','km_automobile','km_cycling','km_total']:
    country_scaled[col+'_pc'] = country_scaled[col] / country_scaled.population.astype(float)

# join country data and mode-shares (WITH TRANSIT) to cities
dfs['data_'+str(year)[2:]]=dfs['data_'+str(year)[2:]].set_index('country').join(country_scaled[['country_num','gdp_standard', 'dependency_standard', 'gasoline_standard','ln_gdp', 'SP.POP.DPND','gasoline_2018']]).reset_index()

In [ ]:
# add in mode-shares INCLUDING TRANSIT to city and country datasets
dfs['data_'+str(year)[2:]]['cycling_share_including_transit'] = dfs['data_'+str(year)[2:]].trips_cycling_touse / dfs['data_'+str(year)[2:]].trips_total_touse
dfs['data_'+str(year)[2:]]['on_foot_share_including_transit'] = dfs['data_'+str(year)[2:]].trips_on_foot_touse / dfs['data_'+str(year)[2:]].trips_total_touse

dfs['data_'+str(year)[2:]] = dfs['data_'+str(year)[2:]].set_index('country')
dfs['data_'+str(year)[2:]] = dfs['data_'+str(year)[2:]].join(country_scaled['on_foot_share_including_transit'], rsuffix='_national')
dfs['data_'+str(year)[2:]] = dfs['data_'+str(year)[2:]].join(country_scaled['cycling_share_including_transit'], rsuffix='_national')
dfs['data_'+str(year)[2:]] = dfs['data_'+str(year)[2:]].join(country_scaled['km_share_on_foot'], rsuffix='_national')
dfs['data_'+str(year)[2:]] = dfs['data_'+str(year)[2:]].join(country_scaled['km_share_cycling'], rsuffix='_national')
dfs['data_'+str(year)[2:]] = dfs['data_'+str(year)[2:]].reset_index()

In [ ]:
dfs['data_'+str(year)[2:]].to_csv('data/data_'+str(year)[2:]+'.csv')
country_scaled.to_csv('data/country_'+str(year)[2:]+'.csv')